In [ ]:
# pip install sentence-transformers

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import pandas as pd
import ast
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.metrics import silhouette_score






In [ ]:
url = "https://docs.google.com/spreadsheets/d/1jAqJJb3870zLfGE1_Xz1cJbnfDkct3lx/export?format=xlsx"

df_vagas = pd.read_excel(url, engine="openpyxl")
df_vagas


,area,vaga,qualidades
0,Jurídico,Paralegal,"Curiosidade, Liderança, Capacidade Analítica"
1,Tecnologia,Engenheiro de Software,"Resiliência, Flexibilidade, Foco em Resultados"
2,Operações,Analista de Processos,"Capacidade Analítica, Pensamento Crítico, Foco..."
3,Agronegócio,Analista Agro,"Comunicação, Capacidade Analítica, Foco em Res..."
4,Consultoria,Consultor Financeiro,"Capacidade Analítica, Autoconfiança, Inteligên..."
...,...,...,...
95,Design,Diretor de Arte,"Adaptabilidade, Trabalho Sob Pressão, Liderança"
96,Educação,Professor,"Tomada de Decisão, Escuta Ativa, Disciplina"
97,Financeiro,Analista de Custos,"Empatia, Escuta Ativa, Inteligência Emocional"
98,Produto,Pesquisador de Produto,"Adaptabilidade, Foco em Resultados, Autoconfiança"


In [ ]:
url1 = "https://docs.google.com/spreadsheets/d/1H8CicVQoYdVmzSGsLgm8LslU4CS5UZLK/export?format=xlsx"

df_pessoas = pd.read_excel(url1)

df_pessoas.head(5)


,nome,idade,area,emprego,qualidades,defeitos,texto,skills_gemini,qualidades_list,defeitos_list,perfil_completo,cluster,cluster_nomeado
0,Lavínia Siqueira,37,Atendimento,Atendente,"Empreendedorismo, Colaboração, Empatia","Apressado, Impulsividade",Atendente por 23 anos,"['Experiência Comprovada', 'Atendimento ao Cli...","['Empreendedorismo', 'Colaboração', 'Empatia']","['Apressado', 'Impulsividade']","Atendimento | Atendente | Empreendedorismo, Co...",2,Cluster Criativo-Comunicacional
1,Srta. Maria Cecília da Mota,59,Jurídico,Advogado,"Criatividade, Empatia, Escuta Ativa","Ansiedade, Ser Crítico Demais",Advogado por 22 anos,"['Experiência Comprovada', 'Conhecimento Juríd...","['Criatividade', 'Empatia', 'Escuta Ativa']","['Ansiedade', 'Ser Crítico Demais']","Jurídico | Advogado | Criatividade, Empatia, E...",4,Cluster Operações & Liderança
2,Emilly Siqueira,53,Logística,Gestor de Armazém,"Foco em Resultados, Visão Sistêmica, Trabalho ...","Rigidez, Insegurança",Gestor de Armazém por 5 anos,"['Liderança Operacional', 'Organização e Plane...","['Foco em Resultados', 'Visão Sistêmica', 'Tra...","['Rigidez', 'Insegurança']",Logística | Gestor de Armazém | Foco em Result...,1,Cluster Humano-Cuidador
3,Juan Farias,43,Operações,Supervisor Operacional,"Trabalho Sob Pressão, Pensamento Crítico, Resp...","Falar Demais, Esquecer Detalhes",Supervisor Operacional por 21 anos,"['Liderança Operacional', 'Ampla Experiência C...","['Trabalho Sob Pressão', 'Pensamento Crítico',...","['Falar Demais', 'Esquecer Detalhes']",Operações | Supervisor Operacional | Trabalho ...,4,Cluster Operações & Liderança
4,Olivia Nunes,41,Vendas,Vendedor,"Pensamento Crítico, Ética, Proatividade","Rigidez, Ansiedade",Vendedor por 4 anos,"['Habilidade de Vendas', 'Atendimento ao Clien...","['Pensamento Crítico', 'Ética', 'Proatividade']","['Rigidez', 'Ansiedade']","Vendas | Vendedor | Pensamento Crítico, Ética,...",2,Cluster Criativo-Comunicacional


Funcao de limpeza

In [ ]:
def clean_text(txt):
    if pd.isna(txt):
        return""
    txt = str(txt)
    txt = re.sub(r"\s+", "", txt).strip()
    return txt

In [ ]:
df_vagas.head()

,area,vaga,qualidades
0,Jurídico,Paralegal,"Curiosidade, Liderança, Capacidade Analítica"
1,Tecnologia,Engenheiro de Software,"Resiliência, Flexibilidade, Foco em Resultados"
2,Operações,Analista de Processos,"Capacidade Analítica, Pensamento Crítico, Foco..."
3,Agronegócio,Analista Agro,"Comunicação, Capacidade Analítica, Foco em Res..."
4,Consultoria,Consultor Financeiro,"Capacidade Analítica, Autoconfiança, Inteligên..."


 LIMPAR COLUNAS IMPORTANTES

In [ ]:
df_vagas["area"] = df_vagas["area"].apply(clean_text)
df_vagas["vaga"] = df_vagas["vaga"].apply(clean_text)
df_vagas["qualidades"] = df_vagas["qualidades"].apply(clean_text)

CRIAR COLUNA PERFIL_VAGA

In [ ]:
df_vagas["perfil_vaga"] = (
    df_vagas["area"] + " | " +
    df_vagas["vaga"] + " | " +
    df_vagas["qualidades"]
)

GERAR EMBEDDINGS DAS VAGAS

In [ ]:
from sentence_transformers import SentenceTransformer



model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

emb_vagas = model.encode(
    df_vagas["perfil_vaga"].tolist(),
    show_progress_bar=True
)

# transformar em dataframe, se quiser salvar
df_emb_vagas = pd.DataFrame(emb_vagas)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df_vagas.head()

,area,vaga,qualidades,perfil_vaga
0,Jurídico,Paralegal,"Curiosidade,Liderança,CapacidadeAnalítica","Jurídico | Paralegal | Curiosidade,Liderança,C..."
1,Tecnologia,EngenheirodeSoftware,"Resiliência,Flexibilidade,FocoemResultados",Tecnologia | EngenheirodeSoftware | Resiliênci...
2,Operações,AnalistadeProcessos,"CapacidadeAnalítica,PensamentoCrítico,FocoemRe...",Operações | AnalistadeProcessos | CapacidadeAn...
3,Agronegócio,AnalistaAgro,"Comunicação,CapacidadeAnalítica,FocoemResultados","Agronegócio | AnalistaAgro | Comunicação,Capac..."
4,Consultoria,ConsultorFinanceiro,"CapacidadeAnalítica,Autoconfiança,Inteligência...",Consultoria | ConsultorFinanceiro | Capacidade...


Apenas salvando

In [ ]:
df_vagas.to_excel("vagas_trabalhadas.xlsx", index=False)
df_emb_vagas.to_excel("embeddings_vagas.xlsx", index=False)

df_vagas.head()

,area,vaga,qualidades,perfil_vaga
0,Jurídico,Paralegal,"Curiosidade,Liderança,CapacidadeAnalítica","Jurídico | Paralegal | Curiosidade,Liderança,C..."
1,Tecnologia,EngenheirodeSoftware,"Resiliência,Flexibilidade,FocoemResultados",Tecnologia | EngenheirodeSoftware | Resiliênci...
2,Operações,AnalistadeProcessos,"CapacidadeAnalítica,PensamentoCrítico,FocoemRe...",Operações | AnalistadeProcessos | CapacidadeAn...
3,Agronegócio,AnalistaAgro,"Comunicação,CapacidadeAnalítica,FocoemResultados","Agronegócio | AnalistaAgro | Comunicação,Capac..."
4,Consultoria,ConsultorFinanceiro,"CapacidadeAnalítica,Autoconfiança,Inteligência...",Consultoria | ConsultorFinanceiro | Capacidade...


In [ ]:
df_pessoas["area"] = df_pessoas["area"].apply(clean_text)
df_pessoas["emprego"] = df_pessoas["emprego"].apply(clean_text)
df_pessoas["qualidades"] = df_pessoas["qualidades"].apply(clean_text)
df_pessoas["texto"] = df_pessoas["texto"].apply(clean_text)

# LIMPAR VAGAS
df_vagas["area"] = df_vagas["area"].apply(clean_text)
df_vagas["vaga"] = df_vagas["vaga"].apply(clean_text)
df_vagas["qualidades"] = df_vagas["qualidades"].apply(clean_text)
df_vagas["perfil_vaga"] = df_vagas["perfil_vaga"].apply(clean_text)

CRIAR PERFIL COMPLETO DAS PESSOAS (se não existir)

In [ ]:
if "perfil_completo" not in df_pessoas.columns:
    df_pessoas["perfil_completo"] = (
        df_pessoas["area"] + " | " +
        df_pessoas["emprego"] + " | " +
        df_pessoas["qualidades"] + " | " +
        df_pessoas["texto"]
    )

 CARREGAR MODELO DE EMBEDDINGS

In [ ]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

GERAR EMBEDDINGS

In [ ]:
print("Gerando embeddings das pessoas...")
emb_pessoas = model.encode(df_pessoas["perfil_completo"].tolist(), show_progress_bar=True)

print("Gerando embeddings das vagas...")
emb_vagas = model.encode(df_vagas["perfil_vaga"].tolist(), show_progress_bar=True)

Gerando embeddings das pessoas...


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Gerando embeddings das vagas...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

CALCULAR SIMILARIDADE

In [ ]:
similaridade = cosine_similarity(emb_pessoas, emb_vagas)
# linhas = pessoas | colunas = vagas

FUNÇÃO PARA PEGAR TOP N VAGAS POR PESSOA

In [ ]:
def recomendar_vagas(idx_pessoa, top_n=5):
    scores = similaridade[idx_pessoa]
    top_idx = scores.argsort()[::-1][:top_n]

    resultados = []
    for i in top_idx:
        resultados.append({
            "vaga": df_vagas.iloc[i]["vaga"],
            "area_vaga": df_vagas.iloc[i]["area"],
            "qualidades_vaga": df_vagas.iloc[i]["qualidades"],
            "score": float(round(scores[i], 4))
        })
    return resultados

GERAR TABELA FINAL DE RECOMENDAÇÕE

In [ ]:
lista_final = []

for i in range(len(df_pessoas)):
    pessoa = df_pessoas.iloc[i]
    recomendadas = recomendar_vagas(i, top_n=5)

    for rec in recomendadas:
        lista_final.append({
            "nome": pessoa["nome"],
            "area_atual": pessoa["area"],
            "emprego_atual": pessoa["emprego"],
            "qualidades_pessoa": pessoa["qualidades"],
            "vaga_sugerida": rec["vaga"],
            "area_vaga": rec["area_vaga"],
            "qualidades_vaga": rec["qualidades_vaga"],
            "similaridade": rec["score"]
        })

df_matches = pd.DataFrame(lista_final)

SALVAR RESULTADOS

In [ ]:
df_matches.to_excel("recomendacoes_pessoas_vagas.xlsx", index=False)

df_matches.head(50)

,nome,area_atual,emprego_atual,qualidades_pessoa,vaga_sugerida,area_vaga,qualidades_vaga,similaridade
0,Lavínia Siqueira,Atendimento,Atendente,"Empreendedorismo,Colaboração,Empatia",CustomerSuccess,Atendimento,"Adaptabilidade,Curiosidade,Ética",0.7538
1,Lavínia Siqueira,Atendimento,Atendente,"Empreendedorismo,Colaboração,Empatia",AnalistadeSuporte,Atendimento,"TrabalhoSobPressão,Flexibilidade,TomadadeDecisão",0.6664
2,Lavínia Siqueira,Atendimento,Atendente,"Empreendedorismo,Colaboração,Empatia",TécnicoAgrícola,Agronegócio,"Comunicação,Criatividade,Persistência",0.6300
3,Lavínia Siqueira,Atendimento,Atendente,"Empreendedorismo,Colaboração,Empatia",GerenteContábil,Contabilidade,"FocoemResultados,Organização,Criatividade",0.6170
4,Lavínia Siqueira,Atendimento,Atendente,"Empreendedorismo,Colaboração,Empatia",AnalistaFiscal,Contabilidade,"TomadadeDecisão,Responsabilidade,Empatia",0.6169
5,Srta. Maria Cecília da Mota,Jurídico,Advogado,"Criatividade,Empatia,EscutaAtiva",Paralegal,Jurídico,"Curiosidade,Liderança,CapacidadeAnalítica",0.7351
6,Srta. Maria Cecília da Mota,Jurídico,Advogado,"Criatividade,Empatia,EscutaAtiva",AnalistaFiscal,Contabilidade,"TomadadeDecisão,Responsabilidade,Empatia",0.5326
7,Srta. Maria Cecília da Mota,Jurídico,Advogado,"Criatividade,Empatia,EscutaAtiva",GerenteContábil,Contabilidade,"FocoemResultados,Organização,Criatividade",0.5176
8,Srta. Maria Cecília da Mota,Jurídico,Advogado,"Criatividade,Empatia,EscutaAtiva",AssistenteContábil,Contabilidade,"PensamentoCrítico,TrabalhoSobPressão,Empatia",0.5013
9,Srta. Maria Cecília da Mota,Jurídico,Advogado,"Criatividade,Empatia,EscutaAtiva",AnalistaFiscal,Contabilidade,"Proatividade,Adaptabilidade,PensamentoCrítico",0.4959


In [ ]:
melhor_match = []

for i in range(len(df_pessoas)):
    scores = similaridade[i]  # linha da matriz = pessoa i
    best_idx = scores.argmax()  # índice da vaga com maior similaridade
    best_score = scores[best_idx]

    melhor_match.append({
        "nome": df_pessoas.iloc[i]["nome"],
        "area_atual": df_pessoas.iloc[i]["area"],
        "emprego_atual": df_pessoas.iloc[i]["emprego"],
        "qualidades_pessoa": df_pessoas.iloc[i]["qualidades"],

        "vaga_recomendada": df_vagas.iloc[best_idx]["vaga"],
        "area_vaga": df_vagas.iloc[best_idx]["area"],
        "qualidades_vaga": df_vagas.iloc[best_idx]["qualidades"],

        "similaridade": round(float(best_score), 4)
    })

df_direcionamento = pd.DataFrame(melhor_match)

df_direcionamento.head(50)


,nome,area_atual,emprego_atual,qualidades_pessoa,vaga_recomendada,area_vaga,qualidades_vaga,similaridade
0,Lavínia Siqueira,Atendimento,Atendente,"Empreendedorismo,Colaboração,Empatia",CustomerSuccess,Atendimento,"Adaptabilidade,Curiosidade,Ética",0.7538
1,Srta. Maria Cecília da Mota,Jurídico,Advogado,"Criatividade,Empatia,EscutaAtiva",Paralegal,Jurídico,"Curiosidade,Liderança,CapacidadeAnalítica",0.7351
2,Emilly Siqueira,Logística,GestordeArmazém,"FocoemResultados,VisãoSistêmica,TrabalhoSobPre...",GestordeArmazém,Logística,"Disciplina,TrabalhoSobPressão,Comunicação",0.7802
3,Juan Farias,Operações,SupervisorOperacional,"TrabalhoSobPressão,PensamentoCrítico,Responsab...",SupervisorIndustrial,Manufatura,"Flexibilidade,Colaboração,TrabalhoSobPressão",0.6838
4,Olivia Nunes,Vendas,Vendedor,"PensamentoCrítico,Ética,Proatividade",Vendedor,Vendas,"EscutaAtiva,Organização,InteligênciaEmocional",0.8189
5,Leandro Abreu,Operações,GestordeProjetos,"CapacidadeAnalítica,Criatividade,Persistência",GestordeProjetos,Operações,"Persistência,Resiliência,Responsabilidade",0.7795
6,Vicente Brito,Atendimento,AnalistadeSuporte,"InteligênciaEmocional,Criatividade,CapacidadeA...",AnalistadeProcessos,Operações,"CapacidadeAnalítica,PensamentoCrítico,FocoemRe...",0.6455
7,Mariana Pacheco,Produto,ProductOwner,"Empatia,TomadadeDecisão,Flexibilidade",ProductManager,Produto,"TomadadeDecisão,Colaboração,Liderança",0.7417
8,Sara Rezende,Vendas,Vendedor,"Criatividade,Responsabilidade,Organização",Vendedor,Vendas,"EscutaAtiva,Organização,InteligênciaEmocional",0.8294
9,Ana Vitória da Conceição,Design,DesignerUX,"Colaboração,Empreendedorismo,Comunicação",DesignerUX,Design,"CapacidadeAnalítica,Comunicação,Liderança",0.7948


In [ ]:
df_direcionamento.to_excel("pessoa_para_vaga_unica.xlsx", index=False)


In [ ]:
print("Gerando embeddings das pessoas...")

model = SentenceTransformer("all-MiniLM-L6-v2")

pessoas_embeddings = model.encode(
    df_pessoas["perfil_completo"].tolist(),
    show_progress_bar=True
)

df_pessoas["embedding"] = list(pessoas_embeddings)

print("Embeddings de pessoas criados com sucesso!")


# ===============================================
# 3) GERAR EMBEDDINGS DAS VAGAS
# ===============================================

print("Gerando embeddings das vagas...")

vagas_embeddings = model.encode(
    df_vagas["perfil_vaga"].tolist(),
    show_progress_bar=True
)

df_vagas["embedding"] = list(vagas_embeddings)

print("Embeddings de vagas criados com sucesso!")


# ===============================================
# 4) FUNÇÃO — RECOMENDAR VAGAS PARA UMA PESSOA
# ===============================================

def recomendar_vagas_para_pessoa(id_pessoa, top_n=5):

    emb_p = np.array(df_pessoas.loc[id_pessoa, "embedding"]).reshape(1, -1)
    vagas_emb = np.vstack(df_vagas["embedding"].values)

    sims = cosine_similarity(emb_p, vagas_emb)[0]
    top_idx = sims.argsort()[::-1][:top_n]

    resultado = df_vagas.iloc[top_idx].copy()
    resultado["similaridade"] = sims[top_idx]

    return resultado[["vaga", "area", "qualidades", "similaridade"]]


# ===============================================
# 5) FUNÇÃO — RECOMENDAR PESSOAS PARA UMA VAGA
# ===============================================

def recomendar_pessoas_para_vaga(id_vaga, top_n=5):

    emb_v = np.array(df_vagas.loc[id_vaga, "embedding"]).reshape(1, -1)
    pessoas_emb = np.vstack(df_pessoas["embedding"].values)

    sims = cosine_similarity(emb_v, pessoas_emb)[0]
    top_idx = sims.argsort()[::-1][:top_n]

    resultado = df_pessoas.iloc[top_idx].copy()
    resultado["similaridade"] = sims[top_idx]

    return resultado[["nome", "area", "emprego", "similaridade"]]


Gerando embeddings das pessoas...


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Embeddings de pessoas criados com sucesso!
Gerando embeddings das vagas...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Embeddings de vagas criados com sucesso!


In [ ]:
def similaridade_entre_pessoas(id1, id2):
    v1 = np.array(df_pessoas.loc[id1, "embedding"]).reshape(1, -1)
    v2 = np.array(df_pessoas.loc[id2, "embedding"]).reshape(1, -1)
    return float(cosine_similarity(v1, v2)[0])

In [ ]:
def validar_matching_pessoa(id_pessoa):
    resultados = recomendar_vagas_para_pessoa(id_pessoa, top_n=10)

    area_pessoa = df_pessoas.loc[id_pessoa, "area"]
    resultados["match_area"] = resultados["area"].apply(lambda a: a == area_pessoa)

    return resultados


In [ ]:
X = np.vstack(df_pessoas["embedding"].values)
labels = df_pessoas["cluster"]

silhouette = silhouette_score(X, labels)
silhouette


np.float32(0.0741485)

In [ ]:
def avaliar_matching_automatico(n=100):
    acertos = []

    for i in range(n):
        pessoa_area = df_pessoas.loc[i, "area"]
        recomendadas = recomendar_vagas_para_pessoa(i, top_n=5)


        score = sum(recomendadas["area"] == pessoa_area)
        acertos.append(score)

    return np.mean(acertos)


In [ ]:
def avaliar_matching_automatico(n=100, top_n=5):
    resultados = []

    total = min(n, len(df_pessoas))

    for i in range(total):
        area_pessoa = df_pessoas.loc[i, "area"]

        recomendadas = recomendar_vagas_para_pessoa(i, top_n=top_n)

        # conta quantas vagas têm a mesma área
        acertos = sum(recomendadas["area"] == area_pessoa)

        resultados.append(acertos)

    return resultados



resultados = avaliar_matching_automatico(n=100, top_n=5)

media = np.mean(resultados)

print("===========================================")
print(f"SCORE MÉDIO DE ACERTO POR PESSOA (0–5): {media:.2f}")
print("===========================================")
print("\nInterpretação:")
print("0.0 → péssimo")
print("1.0 → aceitável")
print("2.0 → muito bom")
print("3.0+ → excelente")
print("5.0 → perfeito")
print("===========================================\n")




SCORE MÉDIO DE ACERTO POR PESSOA (0–5): 2.96

Interpretação:
0.0 → péssimo
1.0 → aceitável
2.0 → muito bom
3.0+ → excelente
5.0 → perfeito



In [ ]:
import pickle

with open("dados_pessoas.pkl", "wb") as f:
    pickle.dump(df_pessoas, f)

with open("dados_vagas.pkl", "wb") as f:
    pickle.dump(df_vagas, f)

print("Arquivos .pkl criados com sucesso!")


Arquivos .pkl criados com sucesso!
